In [ ]:
!pip install seaborn
!pip install keras
!pip install tensorflow

from xml.etree.ElementTree import PI
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings 
import os
import random
import seaborn as sns
import csv
import collections
from keras.preprocessing.image import load_img
from matplotlib.pyplot import figure
from tensorflow.python.client import device_lib
#from tensorflow.keras.utils import to_categorical
from keras import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout
from keras import backend as K
import zipfile
#with zipfile.ZipFile("./petfinder-pawpularity-score/train.zip", 'r') as zip_ref:
    #zip_ref.extractall("./petfinder-pawpularity-score/train")

print(device_lib.list_local_devices())
warnings.filterwarnings('ignore')

mypath = "./petfinder-pawpularity-score"
files = [os.path.splitext(filename)[0] for filename in os.listdir(mypath + "/train/train")]

filetr = open(mypath + "/train.csv")
csvreadertr = csv.reader(filetr)

tr_data = []
for row in csvreadertr:
    tr_data.append((row[0], row[13]))
del tr_data[0]

assert(collections.Counter(list(zip(*tr_data))[0]) == collections.Counter(files))

In [ ]:
df = pd.DataFrame()
df['images'] = files
df['label'] = list(zip(*tr_data))[1]
df['label'] = df['label'].astype(int)

In [ ]:
figure(figsize = (15, 6), dpi = 150)

temp = df['images']
start = random.randint(0, len(temp))
files = temp[start:start+25]

#TODO: figure out how to put ratings beneath each image
ratings = []
for index,file in enumerate(files):    
    plt.subplot(5,5,index+1)
    img = load_img(mypath + "/train/train/" + file + ".jpg")
    img = np.array(img)
    plt.imshow(img)
    plt.axis('off')
    ratings.append(np.array(df['label'][df['images'] == file])[0])

print(ratings)

In [ ]:
figure(figsize = (15, 6), dpi = 150)
sns.set(style="darkgrid")
ax = sns.histplot(df, x = 'label', bins = 100, kde = True)
plt.show()

In [ ]:
#df = df[(df['label'] < 100)]
figure(figsize = (15, 6), dpi = 150)
sns.set(style="darkgrid")
ax = sns.histplot(df, x = 'label', bins = 99, kde = True)
plt.show()

In [ ]:
print(df[df['label'] == 5])
print(df[df['label'] == 100])

In [ ]:
df['images'] = mypath + "/train/train/" + df['images'] + ".jpg" 
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.2, random_state=11)

In [ ]:
print(train[train['label'] == 100])
print(test[test['label'] == 100])

In [ ]:
train = train[(train['label'] < 100) & (train['label'] > 5)]
print(train[train['label'] == 100])
print(test[test['label'] == 100])

In [ ]:
#df['label'] = df['label'].astype('str')
#train['label'] = train['label'].astype('str')
#test['label'] = test['label'].astype('str')

In [ ]:
print(train['label'])

In [ ]:
# Create data generator for the images
# splitting into testing and training data
from keras.preprocessing.image import ImageDataGenerator
train_generator = ImageDataGenerator(
    rescale = 1./255, # normalize the image 
    rotation_range = 360, # augmentation of images (helps to avoid overfitting)
    width_shift_range=0.35,
    height_shift_range=0.35,
    shear_range = 0.35,
    zoom_range = 0.35,
    horizontal_flip = True,
    vertical_flip = True,
    fill_mode = 'nearest',
)

val_generator = ImageDataGenerator(rescale=1./255)

# TODO: can't seem to find documentation on what target_size does under hood
# from source code, the 'dataframe' argument is the dataframe of full image paths
train_iterator = train_generator.flow_from_dataframe(
    train, 
    x_col='images', 
    y_col='label', 
    target_size=(128,128), 
    batch_size=64, 
    class_mode='raw'
)

val_iterator = val_generator.flow_from_dataframe(
    test, 
    x_col='images', 
    y_col='label', 
    target_size=(128,128), 
    batch_size=64, 
    class_mode='raw'
)

#print(np.array(train_iterator[123][0]).shape)
#train_iterator[0][1][0][0][0]
testflat = np.array(train_iterator[0][0])
print(testflat.shape)
#print(testflat_f.shape)

counter = 0
stack = None
super = None
for s in train_iterator:    
    testflat = s[0]
    if (counter == 3000):
        break
    for b in testflat:
        #counter = counter + 1
        if (stack is None):
            stack = b.flatten()
        else: stack = np.vstack((stack, b.flatten()))
        print(stack.shape)
    if (super is None):
            super = stack
    else: super = np.vstack((super, stack))
    counter = counter + 1


In [ ]:
x_np_array = np.zeros(7502*128*128*3)
x_np_array = np.reshape(x_np_array, (7502, 49152))
#print(x_np_array.shape)
#print(x_np_array)

y_np_array = np.zeros(7502)
y_np_array = np.reshape(y_np_array, (7502, 1))
print(y_np_array.shape)

In [ ]:
xv_np_array = np.zeros(1983*128*128*3)
xv_np_array = np.reshape(xv_np_array, (1983, 49152))
#print(x_np_array.shape)
#print(x_np_array)

yv_np_array = np.zeros(1983)
yv_np_array = np.reshape(yv_np_array, (1983, 1))
print(yv_np_array.shape)

In [ ]:
counter = 0

for s in train_iterator:    
    if counter == 7502:
        break
    batch = s[0]
    batch_labels = s[1]
    
    #print(batch.shape)
    #print(batch_labels.shape)
    #print(x_np_array.shape)
    cardinality = batch.shape[0]       
    print("batch size: " + str(cardinality))

    for c in range(cardinality):
        print("c: " + str(c))
        x_np_array[counter] = batch[c].flatten()
        y_np_array[counter] = batch_labels[c]             
        print("counter: " + str(counter))
        counter = counter + 1    

#print(y_np_array)

In [ ]:
counter = 0

for s in val_iterator:    
    if counter == 1983:
        break

    batch = s[0]
    batch_labels = s[1]
   
    cardinality = batch.shape[0]       
    print("batch size: " + str(cardinality))

    for c in range(cardinality):
        print("c: " + str(c))
        xv_np_array[counter] = batch[c].flatten()
        yv_np_array[counter] = batch_labels[c]             
        print("counter: " + str(counter))
        counter = counter + 1    

#print(y_np_array)

In [ ]:
#print(np.sum(y_np_array))
#np.savetxt("pawp_train_flat.csv", arr_re, delimiter=",")

In [ ]:
plt.imshow(x_np_array[0].reshape((128, 128, 3)))

In [ ]:
from sklearn.svm import SVR
print("foo")
svr_rbf = SVR(kernel='poly', C=0.2, gamma='scale', epsilon=0.1, tol=0.001, degree=5)
fit = svr_rbf.fit(x_np_array, y_np_array)

In [ ]:
print("bar")

In [ ]:
y_rbf = fit.predict(x_np_array)

In [ ]:
plt.hist(y_np_array)

In [ ]:
plt.hist(y_rbf)
yv = fit.predict(xv_np_array)

In [ ]:
from sklearn.metrics import mean_squared_error
rms = mean_squared_error(xv_np_array, y_v, squared=False)
print(rms)


model2 = Sequential([
    Conv2D(64, (7,7), activation='relu', input_shape=(128,128,3)),
    MaxPool2D((3,3)),
    Conv2D(16, (5,5), activation='relu'),
    MaxPool2D((3,3)),
    Conv2D(16, (4,4), activation='relu'),
    Conv2D(16, (2,2), activation='relu'),
    #Conv2D(256, (2,2), activation='relu'),
    MaxPool2D((2,2)),
    Flatten(),
    Dropout(rate = 0.25),    
    Dense(2048, activation='sigmoid'),    
    Dense(1, activation='linear'),
])

In [ ]:
plt.hist(yv_np_array)


In [ ]:
plt.hist(yv)

In [ ]:
from sklearn.svm import SVR
svr_rbf = SVR(kernel='poly', C=0.2, gamma='scale', epsilon=0.5, tol=0.001, degree=5)
fit = svr_rbf.fit(x_np_array, y_np_array)
y_rbf = fit.predict(x_np_array)
plt.hist(y_np_array)



In [ ]:
plt.hist(y_rbf)
yv = fit.predict(xv_np_array)
rms = mean_squared_error(xv_np_array, y_v, squared=False)
print(rms)

In [ ]:
plt.hist(yv_np_array)


In [ ]:
plt.hist(yv)

In [ ]:
from sklearn.svm import SVR
svr_rbf = SVR(kernel='poly', C=0.2, gamma='scale', epsilon=0.1, tol=0.001, degree=10)
fit = svr_rbf.fit(x_np_array, y_np_array)
y_rbf = fit.predict(x_np_array)
plt.hist(y_np_array)


In [ ]:
plt.hist(y_rbf)
yv = fit.predict(xv_np_array)
rms = mean_squared_error(xv_np_array, y_v, squared=False)
print(rms)

In [ ]:
plt.hist(yv_np_array)


In [ ]:
plt.hist(yv)

In [ ]:
from sklearn.svm import SVR
svr_rbf = SVR(kernel='poly', C=0.2, gamma='scale', epsilon=0.5, tol=0.001, degree=10)
fit = svr_rbf.fit(x_np_array, y_np_array)
y_rbf = fit.predict(x_np_array)
plt.hist(y_np_array)


In [ ]:
plt.hist(y_rbf)
yv = fit.predict(xv_np_array)
rms = mean_squared_error(xv_np_array, y_v, squared=False)
print(rms)

In [ ]:
plt.hist(yv_np_array)


In [ ]:
plt.hist(yv)

In [ ]:
from sklearn.svm import SVR
svr_rbf = SVR(kernel='poly', C=0.1, gamma='scale', epsilon=0.1, tol=0.001, degree=10)
fit = svr_rbf.fit(x_np_array, y_np_array)
y_rbf = fit.predict(x_np_array)
plt.hist(y_np_array)

In [ ]:
plt.hist(y_rbf)
yv = fit.predict(xv_np_array)
rms = mean_squared_error(xv_np_array, y_v, squared=False)
print(rms)

In [ ]:
plt.hist(yv_np_array)


In [ ]:
plt.hist(yv)

In [ ]:
from sklearn.svm import SVR
svr_rbf = SVR(kernel='poly', C=0.1, gamma='scale', epsilon=0.5, tol=0.001, degree=10)
fit = svr_rbf.fit(x_np_array, y_np_array)
y_rbf = fit.predict(x_np_array)
plt.hist(y_np_array)



In [ ]:
plt.hist(y_rbf)
yv = fit.predict(xv_np_array)
rms = mean_squared_error(xv_np_array, y_v, squared=False)
print(rms)

In [ ]:
plt.hist(yv_np_array)


In [ ]:
plt.hist(yv)

In [ ]:
from sklearn.svm import SVR
svr_rbf = SVR(kernel='poly', C=0.1, gamma='scale', epsilon=0.1, tol=0.001, degree=7)
fit = svr_rbf.fit(x_np_array, y_np_array)
y_rbf = fit.predict(x_np_array)
plt.hist(y_np_array)


In [ ]:
plt.hist(y_rbf)
yv = fit.predict(xv_np_array)
rms = mean_squared_error(xv_np_array, y_v, squared=False)
print(rms)

In [ ]:
plt.hist(yv_np_array)


In [ ]:
plt.hist(yv)

In [ ]:
from sklearn.svm import SVR
svr_rbf = SVR(kernel='poly', C=0.2, gamma='scale', epsilon=0.5, tol=0.001, degree=7)
fit = svr_rbf.fit(x_np_array, y_np_array)
y_rbf = fit.predict(x_np_array)
plt.hist(y_np_array)

In [ ]:
plt.hist(y_rbf)
yv = fit.predict(xv_np_array)
rms = mean_squared_error(xv_np_array, y_v, squared=False)
print(rms)

In [ ]:
plt.hist(yv_np_array)


In [ ]:
plt.hist(yv)